# Relational Databases
- PostgreSQL
- SQL server
- Oracle Database
- Oracle MySQL
- MariaDB
- SQlite
- ...

Standard SQL

1+ connector by vendor|provider following the same model PEP 248 (DB API V1) ou PEP 249 (DB API v2)
- postgresql: psycopg2|3
- sql server: pyodbc ou pymssql
- oracle db: cx_oracle

ORM SQLAlchemy (Object Relational Mapper)

In [1]:
import psycopg2 as pg
import pandas as pd
from sqlalchemy import create_engine

## DB API (low level)

In [3]:
host = 'localhost'
port = 5432
dbname = 'dbcity'
user = 'citymngr'  # postgres
password = 'password'

In [5]:
url = f"host={host} port={port} dbname={dbname} user={user} password={password}"

In [37]:
conn = pg.connect(url)
conn

<connection object at 0x0000011CD0BC6AC0; dsn: 'user=citymngr password=xxx dbname=dbcity host=localhost port=5432', closed: 0>

In [38]:
conn.close()

In [40]:
# use with a context manager
with pg.connect(url) as conn1:
    print(conn1.closed)
print(conn1.closed)    

0
0


In [59]:
conn = pg.connect(url)
cursor = conn.cursor()
cursor.execute('SELECT * FROM city')
data = list(cursor)
print(data)
cursor.close()
conn.close()

[(2, None, None, '31000', 'Toulouse', None, None, None, '31', None, None), (3, None, None, '64000', 'Pau', None, None, None, '64', None, None)]


In [ ]:
conn = pg.connect(url)
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("INSERT INTO city (label, zip_code, department_number) VALUES ('Toulouse','31000', '31')")
cursor.close()
conn.close()

In [56]:
name = 'Pau'
zipcode = '64000'
dept = '64'

In [58]:
with pg.connect(url) as conn:
    conn.autocommit = True
    with conn.cursor() as cursor:
        cursor.execute("INSERT INTO city (label, zip_code, department_number) VALUES (%s, %s, %s)", (name, zipcode, dept))

In [60]:
with pg.connect(url) as conn:
    with conn.cursor() as cursor:
        cursor.execute('SELECT * FROM city')
        data = list(cursor)
data

[(2, None, None, '31000', 'Toulouse', None, None, None, '31', None, None),
 (3, None, None, '64000', 'Pau', None, None, None, '64', None, None)]

## Pandas et SQLAlchemy

In [4]:
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

In [64]:
with engine.connect():
    pass

In [38]:
mssql_user='sa'
mssql_password='Passw@rd'
mssql_port = 1433
#engine2 = create_engine(f"mssql+pymssql://{mssql_user}:{mssql_password}@{host}:{mssql_port}/{dbname}")
engine2 = create_engine(f"mssql+pyodbc://{mssql_user}:{mssql_password}@{host}:{mssql_port}/{dbname}?driver=ODBC+Driver+2022+for+SQL+Server")
#"mssql+pyodbc://{user}:{password}@{host}/{dbname}?driver=ODBC+Driver+17+for+SQL+Server

In [40]:
# NB: need to configure odbc
# with engine2.connect():
#     pass

In [6]:
dfCities = pd.read_sql_table('city', con=engine)
dfCities

,id,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,2,None,None,31000,Toulouse,NaN,NaN,None,31,None,None
1,3,None,None,64000,Pau,NaN,NaN,None,64,None,None


In [8]:
dfCities = pd.read_csv('data/cities/cities.csv', encoding='UTF-8', dtype={'zip_code':str})
dfCities.head()

,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,25620,ville du pont,25650,ville du pont,46.999873,6.498147,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
1,25624,villers grelot,25640,villers grelot,47.361512,6.235167,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
2,25615,villars les blamont,25310,villars les blamont,47.368384,6.871415,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
3,25619,les villedieu,25240,les villedieu,46.713906,6.265831,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
4,25622,villers buzon,25170,villers buzon,47.228558,5.852187,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté


In [11]:
# ValueError: Table 'city' already exists.
dfCities.to_sql('city2', con=engine)

145

In [13]:
dfCities.to_sql('city', con=engine, if_exists='append', index=False)

145

In [14]:
len(dfCities)

39145

In [23]:
dept = '64'

# avec pandas ramener les villes du département dpt
sql = "SELECT * FROM city WHERE department_number = %s"
dfCitiesMyDept = pd.read_sql_query(sql, con=engine, params=(dept,))
dfCitiesMyDept

,id,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,3,None,None,64000,Pau,NaN,NaN,None,64,None,None
1,16348,64001,aast,64460,aast,43.291176,-0.081741,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
2,16351,64003,abidos,64150,abidos,43.395117,-0.627969,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
3,16354,64006,accous,64490,accous,42.923529,-0.613590,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
4,16357,64007,agnos,64400,agnos,43.153436,-0.630451,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
...,...,...,...,...,...,...,...,...,...,...,...
561,18783,64529,sus,64190,sus,43.291746,-0.780735,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
562,18786,64540,urcuit,64990,urcuit,43.485045,-1.350222,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
563,18789,64558,villefranque,64990,villefranque,43.441684,-1.447768,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine
564,18792,64559,viodos abense de bas,64130,viodos abense de bas,43.247703,-0.896421,pyrénées-atlantiques,64,nouvelle-aquitaine,Nouvelle-Aquitaine


In [36]:
# Not accepted by psycopg2 (by default)
dept = '64'

# avec pandas ramener les villes du département dpt
sql = "SELECT * FROM city WHERE department_number = :dept"
try: 
    dfCitiesMyDept = pd.read_sql_query(sql, con=engine, params={'dept': dept})
    print(dfCitiesMyDept)
except Exception as ex:
    print('no arg by name with this engine:', type(ex))
    print('detail:', ex)

no arg by name with this engine: <class 'sqlalchemy.exc.ProgrammingError'>
detail: (psycopg2.errors.SyntaxError) syntax error at or near ":"
LINE 1: SELECT * FROM city WHERE department_number = :dept
                                                     ^

[SQL: SELECT * FROM city WHERE department_number = :dept]
[parameters: {'dept': '64'}]
(Background on this error at: https://sqlalche.me/e/20/f405)
